In [1]:
# install HF transformers
!pip install transformers

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [2]:
# uodate the xformers package
!pip install -U xformers

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
# install package to read pdf tabular data
!pip install tabula-py

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [4]:
# install package to read pdf
!pip install pypdf

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [6]:
# install nltk package
!pip install nltk

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [1]:
# import relevant packages
import tabula
import pandas as pd
import numpy as np
import PyPDF2 
from pypdf import PdfReader
import textract
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from transformers import pipeline

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# create a variable for the file to be loaded
file = "Recommendation Letter-Fong Weng Cheun (signed version).pdf"

In [3]:
# write a function to extract text should there be more than 1 page
def extract_text_with_pyPDF(pdf_file):
    pdf_reader = PdfReader(pdf_file)
    raw_text = ''
    for i, page in enumerate(pdf_reader.pages):
        text = page.extract_text()
        if text:
            raw_text += text
    return raw_text

In [4]:
# run the function and print the result
text = extract_text_with_pyPDF(file)

In [5]:
# let's have a look at the extracted text
print(text[0:3027])

 
 
 
 
June 1st, 2023 
 
 
RECOMMENDATION LETTER FOR MR. FONG  WENG CHEUN  
 
Dear Sir/Madam , 
 
We are very pleased to recommendation for Mr. Fong Weng Cheun , who has been an exceptional 
employee at ERNI Asia Holding Pte. Ltd . / Tyco Electronics Singapore Pte. Ltd. for over four years. 
After the acquisition of ERNI by TE Connectivity  in early September 2021, we have had the pleasure 
of working with and supervising Fong and have witnessed his outstanding contributions to the 
organization  / region. 
 
Fong has consistently demonstrated exemplary skills and dedication in his role, delivering remarkable 
results across various critical projects. Notably, he actively participated in systems migration from 
NAV 2013 to NAV 2018 and SAP, ensuring a seamless transition and enhancing operational 
efficiency. The meticulous planning, attention to detail, and ability to collaborate with cross -functional 
teams were instrumental in the project's success.  
 
Furthermore, Fong played a 

In [8]:
#The word_tokenize() function will break our text phrases into individual words.
tokens = word_tokenize(text)
#We'll create a new list that contains punctuation we wish to clean.
punctuations = ['(',')',';',':','[',']',',']
#We initialize the stopwords variable, which is a list of words like "The," "I," "and," etc. that don't hold much value as keywords.
stop_words = stopwords.words('english')
#We create a list comprehension that only returns a list of words that are NOT IN stop_words and NOT IN punctuations.
keywords = [word for word in tokens if not word in stop_words and not word in punctuations]

In [9]:
parse_text = text[:3027]
parse_text

" \n \n \n \nJune 1st, 2023 \n \n \nRECOMMENDATION LETTER FOR MR. FONG  WENG CHEUN  \n \nDear Sir/Madam , \n \nWe are very pleased to recommendation for Mr. Fong Weng Cheun , who has been an exceptional \nemployee at ERNI Asia Holding Pte. Ltd . / Tyco Electronics Singapore Pte. Ltd. for over four years. \nAfter the acquisition of ERNI by TE Connectivity  in early September 2021, we have had the pleasure \nof working with and supervising Fong and have witnessed his outstanding contributions to the \norganization  / region. \n \nFong has consistently demonstrated exemplary skills and dedication in his role, delivering remarkable \nresults across various critical projects. Notably, he actively participated in systems migration from \nNAV 2013 to NAV 2018 and SAP, ensuring a seamless transition and enhancing operational \nefficiency. The meticulous planning, attention to detail, and ability to collaborate with cross -functional \nteams were instrumental in the project's success.  \n \nFur

In [10]:
# write a fucntion to breakdown the text to smaller chuncks so that they could be applied to the model for the classification / sentiment-analysis task due to size limitation
def split_text(text):
    max_chunk_size = 512
    chunks = []
    current_chunk = ""
    for sentence in text.split("."):
        if len(current_chunk) + len(sentence) < max_chunk_size:
            current_chunk += sentence + "."
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + "."
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

In [11]:
# activate the function to split the extracted text into smaller chunks
chunk = split_text(parse_text)

In [12]:
# let's have a look at the 1st chunk of the extracted text
chunk[0]

'June 1st, 2023 \n \n \nRECOMMENDATION LETTER FOR MR. FONG  WENG CHEUN  \n \nDear Sir/Madam , \n \nWe are very pleased to recommendation for Mr. Fong Weng Cheun , who has been an exceptional \nemployee at ERNI Asia Holding Pte. Ltd . / Tyco Electronics Singapore Pte. Ltd. for over four years. \nAfter the acquisition of ERNI by TE Connectivity  in early September 2021, we have had the pleasure \nof working with and supervising Fong and have witnessed his outstanding contributions to the \norganization  / region.'

In [13]:
# what's the analysis of this chunk by the model?
classifier = pipeline(model="distilbert-base-uncased-finetuned-sst-2-english")
classifier(chunk[0])

WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.0.1+cu118 with CUDA 1108 (you have 2.0.1)
    Python  3.10.11 (you have 3.10.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


[{'label': 'POSITIVE', 'score': 0.9997197985649109}]

The model is 99.97% confident that this chunk of text exhibits positive sentiment

In [14]:
# let's continue the process for the rest
print(classifier(chunk[1]))
print(classifier(chunk[2]))
print(classifier(chunk[3]))
print(classifier(chunk[4]))
print(classifier(chunk[5]))
print(classifier(chunk[6]))

[{'label': 'POSITIVE', 'score': 0.9998272061347961}]
[{'label': 'POSITIVE', 'score': 0.999442994594574}]
[{'label': 'POSITIVE', 'score': 0.9997065663337708}]
[{'label': 'POSITIVE', 'score': 0.9998099207878113}]
[{'label': 'POSITIVE', 'score': 0.9993143081665039}]
[{'label': 'POSITIVE', 'score': 0.9997186064720154}]


Overall, the model analyzed that the recommendation letter exhibits positivity on work performance, with a high confidence score.

In [15]:
# what's the length of the recommendation letter?
len(parse_text)

3027

In [16]:
# let's summarize to a shorter version
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
response = summarizer(parse_text, min_length=100, max_length=250)
print(response)

[{'summary_text': ' Fong Weng Cheun has been an exceptional employee at ERNI Asia Holding Pte. Ltd. for over four years . Fong showcased exceptional leadership and adaptability in facilitating smooth business transfers in Japan, K orea, China, and Singapore . He was key to align to implement TE ’s fina ncial po licies , and c onsistently provide d high quality  \xa0information in a timely manner and with minimal supervision . We have no hesitation in recommending Fong for any future endeavors . We are confi dent that with h is dedication, expertise,  \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0and ability to deliver results Fong will be a valuable asset to any future employer .'}]
